In [1]:
from langchain_community.document_loaders import YoutubeLoader
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown, Latex
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from crewai_tools import BaseTool

load_dotenv()

model_name = os.getenv("OPENAI_MODEL_NAME", "gpt-4o-mini")
llm = ChatOpenAI(temperature=0, model_name=model_name)

In [5]:
 import urllib
import warnings

from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

warnings.filterwarnings("ignore")

class TranscriptVideo(BaseTool):
    name: str = "Transcrever Vídeo"
    description: str = (
        "Transcreve o áudio de um vídeo do YouTube."
    )

    def _run(self, url_video: str) -> str:
        llm = ChatOpenAI(temperature=0, model_name=model_name)

        loader = YoutubeLoader.from_youtube_url(
            url_video,
            add_video_info=True,
            language=["pt","en"],
        )
        document = loader.load()

        question_prompt_template = """
                        Forneça um resumo em tópicos dos assuntos tratados no texto a seguir.
                        TEXTO: {text}
                        RESUMO:
                        """

        question_prompt = PromptTemplate(
            template=question_prompt_template, input_variables=["text"]
        )

        refine_prompt_template = """
                    Escreva um resumo conciso do texto a seguir delimitado por crases triplas.
                    Retorne sua resposta em tópicos que cubram todos os pontos-chave do texto.
                    ```{text}```
                    RESUMO DOS PONTOS:
                    """

        refine_prompt = PromptTemplate(
            template=refine_prompt_template, input_variables=["text"]
        )

        refine_chain = load_summarize_chain(
            llm,
            chain_type="refine",
            question_prompt=question_prompt,
            refine_prompt=refine_prompt,
            return_intermediate_steps=True,
        )
        refine_outputs = refine_chain({"input_documents": document})

        return refine_outputs["output_text"]


In [6]:
from crewai import Agent, Crew, Process, Task

# abrir arquivo yaml como dict
def load_yaml(file_path):
    import yaml
    with open(file_path, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

agents_config = load_yaml("src/youtube_sommelier/config/agents.yaml")
tasks_config = load_yaml("src/youtube_sommelier/config/tasks.yaml")

In [7]:
video_transcriber = Agent(
	config=agents_config['video_transcriber'],
    tools=[TranscriptVideo(result_as_answer=True)],
    verbose=True,
    llm=llm,
	allow_delegation=False,
)

transcript_video = Task(
	config=tasks_config['transcript_video'],
    agent=video_transcriber,
)

crew = Crew(
  agents=[video_transcriber],
  tasks=[transcript_video],
  verbose=2,
  memory=False,
  llm=llm,
  full_output=True,
  planning=False,
)

input_text  = "Gostaria de transcrever o áudio de um vídeo do YouTube. https://www.youtube.com/watch?v=VsZXAEFKXC4"

inputs = {
    "topic": input_text
}
result = crew.kickoff(inputs=inputs)

2024-07-31 23:47:10,862 - 138198914737216 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-07-31 23:47:10][DEBUG]: == Working Agent: Transcritor de vídeo

 [2024-07-31 23:47:10][INFO]: == Starting Task: Entrada: Gostaria de transcrever o áudio de um vídeo do YouTube. https://www.youtube.com/watch?v=VsZXAEFKXC4 Verifique se na entrada informada existe um vídeo. Se sim, Crie uma transcrição  completa do conteúdo do vídeo. A transcrição deve ser precisa.



> Entering new CrewAgentExecutor chain...
Action: Transcrever Vídeo  
Action Input: {"url_video": "https://www.youtube.com/watch?v=VsZXAEFKXC4"}   

### Resumo em Tópicos

1. **Introdução ao Vídeo**
   - Discussão sobre a necessidade de polêmica para atrair visualizações.
   - Apresentação do programa "Dois Dedos de Teologia".

2. **Ambiente da Entrevista**
   - Gravação em uma clínica de fisioterapia.
   - Entrevista enquanto se realiza um treino de pernas.

3. **Convidado Especial**
   - Julinho da banda Purple é o convidado.
   - Conversa sobre sua trajetória musical e vida cristã.

4. **Experiência Musical de Juli

In [8]:
Markdown(result.raw)

### Resumo em Tópicos

1. **Introdução ao Vídeo**
   - Discussão sobre a necessidade de polêmica para atrair visualizações.
   - Apresentação do programa "Dois Dedos de Teologia".

2. **Ambiente da Entrevista**
   - Gravação em uma clínica de fisioterapia.
   - Entrevista enquanto se realiza um treino de pernas.

3. **Convidado Especial**
   - Julinho da banda Purple é o convidado.
   - Conversa sobre sua trajetória musical e vida cristã.

4. **Experiência Musical de Julinho**
   - Crescimento em um ambiente cristão desde pequeno.
   - Conversão em 2006 durante um acampamento de carnaval.
   - Início na música tocando bateria e depois cantando.

5. **Formação da Banda Purple**
   - Banda sempre existiu informalmente, mas se reuniu oficialmente em 2015.
   - Discussão sobre a evolução e o repertório da banda.

6. **Desafios do Ministério Musical**
   - Dilema entre ser uma banda de ministério e as exigências do mercado.
   - Necessidade de profissionalização e estrutura para apresentações.

7. **Relação com a Igreja**
   - Importância de adaptar o repertório para o público da igreja.
   - A música como ferramenta de louvor e edificação.

8. **Críticas e Expectativas**
   - Enfrentamento de críticas sobre a profissionalização do ministério.
   - Reflexão sobre a percepção de que o ministério deve ser gratuito.

9. **Importância da Comunidade**
   - Valorização de um círculo de apoio que não se impressiona com a fama.
   - Necessidade de manter a humildade e a conexão com a vida cotidiana.

10. **Reflexões Finais**
    - Discussão sobre a identidade além do ministério.
    - A importância de continuar obedecendo ao chamado de Deus.

11. **Encerramento**
    - Agradecimento ao convidado e convite para seguir nas redes sociais.